<a href="https://colab.research.google.com/github/ngupta23/medium_articles/blob/main/ts_2_linear_vision_pycaret_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here is the original notebook which is converted into a PyCaret flow in this notebook: https://www.kaggle.com/konradb/ts-2-linear-vision**

This is part of a series of notebooks about practical time series methods:
* [Part 0: the basics](https://www.kaggle.com/konradb/ts-0-the-basics)
* [Part 1a: smoothing methods](https://www.kaggle.com/konradb/ts-1a-smoothing-methods)
* [Part 1b: Prophet](https://www.kaggle.com/konradb/ts-1b-prophet) 
* [Part 2: ARIMA](https://www.kaggle.com/konradb/ts-2-arma-and-friends) - this notebook

The series is accompompanied by video presentations on the [Abhishek](https://www.kaggle.com/abhishek)  YouTube channel:
* [Talk 0: presenting the basics](https://www.youtube.com/watch?v=cKzXOOtOXYY) - based on content from part0 mostly
* [Talk 1: curve fitting](https://www.youtube.com/watch?v=kAI67Sz92-s) - combining the content from parts 1a and 1b
* [Talk 2: ARIMA and friends](https://www.youtube.com/watch?v=kAI67Sz92-s) - ARIMA models

---------------------------------------

The notebook is split into three sections: we introduce the basic framework of linear processes, then present extensions and finally demonstrate how to solve a prediction problem from scratch. 

* [Basic linear processes](#section-one)
* [Beyond ARMA](#section-two)
* [Full pipeline](#section-three)
* [A day at the races](#section-four)

As usual, we begin by importing the required packages

In [1]:
def what_is_installed():
    import pycaret
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except:
    !pip install prophet
    !pip install pycaret-ts-alpha
    what_is_installed()


System:
    python: 3.7.12 (default, Jan 15 2022, 18:48:18)  [GCC 7.5.0]
executable: /usr/bin/python3
   machine: Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic

Python dependencies:
          pip: 21.1.3
   setuptools: 57.4.0
      pycaret: 3.0.0
      sklearn: 1.0.2
       sktime: 0.10.1
  statsmodels: 0.12.2
        numpy: 1.21.5
        scipy: 1.7.3
       pandas: 1.3.5
   matplotlib: 3.2.2
       plotly: 5.5.0
       joblib: 1.0.1
        numba: 0.55.1
       mlflow: 1.24.0
     lightgbm: 3.3.2
      xgboost: 0.90
     pmdarima: 1.8.5
        tbats: Installed but version unavailable
      prophet: 1.0
      tsfresh: Not installed


In [2]:
# # It's a Surprise Tool That Will Help Us Later ;-)
# !pip install pmdarima

#############################################################
#### NOTE: No need to install separately for PyCaret !!! ####
#############################################################

In [3]:
#################################################################
#### NOTE: Most of these imports are not needed.             ####
#### This is managed by PyCaret internally automatically !!! ####
#################################################################
from pycaret.time_series import TSForecastingExperiment

# import itertools
import pandas as pd
import numpy as np
# from random import gauss


# import statsmodels.api as sm
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_process import ArmaProcess

# from pandas.plotting import autocorrelation_plot
# from sklearn.metrics import mean_squared_error

# import os
# from IPython.display import Image

# import matplotlib.pyplot as plt
# import warnings
# import itertools
# import statsmodels.api as sm
# from random import gauss
# from pandas.plotting import autocorrelation_plot
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# from pmdarima.arima import auto_arima
# from pmdarima import pipeline
# from pmdarima import model_selection
# from pmdarima import preprocessing as ppc
# from pmdarima import arima

# from fbprophet import Prophet 

# import matplotlib.pyplot as plt
# plt.style.use('fivethirtyeight') 

# warnings.simplefilter(action='ignore', category= FutureWarning)

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
#### Not needed for PyCaret

# # general settings
# class CFG:
#     data_folder = '../input/tsdata-1/'
#     img_dim1 = 20
#     img_dim2 = 10
        
# # adjust the parameters for displayed figures    
# plt.rcParams.update({'figure.figsize': (CFG.img_dim1,CFG.img_dim2)})    

<a id="section-one"></a>
# Basic linear processes

Until 1980s, ARIMA processes were the workhorses of time series analysis, which for many people meant just finding right order of such model. This class of processes is defined through linear relationship between observation and noise factors.

Just to indulge my inner nerd, I will go top-down in this one and start from a high level definition - because why not :-) A linear process is a time series $X_t$ defined by:

\begin{equation}
    X_t = \sum_{u = -\infty}^{\infty} \psi_u \epsilon_{t-u}
\end{equation}

where $\epsilon_t$ is a white noise series and 

\begin{equation}
\sum_{u = -\infty}^{\infty} \left|\psi_u \right|^2 < \infty
\end{equation}


Translated to plain English:
* (in our context) white noise a serially uncorrelated sequence of random variables with zero mean and finite variance. For a general definition (and the origin of the color convention) I can recommend the Wikipedia entry: https://en.wikipedia.org/wiki/White_noise

* it is a linear combination - potentially infinite and depending on both past and present - of a white noise series

* in the real world it has to be causal $\implies$ depending on a finite number of past values

* it is a unified framework for handling different types of data generating processes

* for the mathematically inclided person reading this: huge parts of the theory behind ARMA processes pop up as special cases / corrolaries from results in functional analysis, specifically the [Hilbert projection theorem](https://en.wikipedia.org/wiki/Hilbert_projection_theorem) and [spectral theorem](https://en.wikipedia.org/wiki/Spectral_theorem); the requirement on sum of squared values being finite correspond to belonging to an $l_2$ space for our sequence: https://en.wikipedia.org/wiki/Lp_space

* only needs second order statistics (variance) $\rightarrow$ works best for elliptical distributions (e.g. Gaussian)



## AR processes

The simplest (non-trivial) example of a linear process is an autoregressive process of order $p$:

\begin{equation}
    X_t = \sum_{i= 1}^p \phi_i X_{t-i} + \epsilon_t
\end{equation}
    
Unpacking the formula:
1. Natural extension of a multiple linear regression model: we forecast the variable of interest using a linear combination of past values of the variable (lagged values act as predictors). The term \textit{auto}regression reflects the fact that we are regressing the variable on (version of) itself
2. The single exponential smoothing method (described [here](https://www.kaggle.com/konradb/ts-1a-smoothing-methods) ) can be viewed as a special case of an autoregressive process of order 1
3. Easy to estimate parameters and forecast 


Now that we have defined (and hopefully understood) what an AR process is, let's have a look at how to identify one. This is relevant if we need to decide whether an autoregressive process is a right kind of model for a particular dataset. For the sake of demonstration, we will use simulated data, where the data generating process is:

\begin{equation}
    X_t = 0.9 X_{t-1} + \epsilon_t
\end{equation}
    
Observe in the codeblock below that the coefficient changes sign - this is because of the notational convention in statsmodels, where we read the coefficients in order defined by the characteristic polynomial (more on that below):

\begin{equation}
    X_t - 0.9 X_{t-1} = \epsilon_t
\end{equation}


A fast technique for deciding if AR is the right kind of model for a given series is to examine the autocorrelation and partial autocorrelation functions (described [here](https://www.kaggle.com/konradb/practical-time-series-pt-1-the-basics#Dependence)). 

In [5]:
ar1 = np.array([1, -0.9])
ma1 = np.array([1])
AR_object1 = ArmaProcess(ar1, ma1)
simulated_data_1 = pd.Series(AR_object1.generate_sample(nsample=1000))

# #### Using Manual Approach ----
# plt.plot(simulated_data_1)
# plt.show()

# plot_acf(simulated_data_1, lags = 10)
# plt.show()

# plot_pacf(simulated_data_1, lags = 10)
# plt.show()

################################
#### Using PyCaret Approach ####
################################
# Change renderer appropriately based on where the notebook is being run ----
# Refer to plotly for available renderers.
global_plot_settings = {"renderer": "colab", "hoverinfo": "text"}

exp = TSForecastingExperiment()
exp.setup(data = simulated_data_1, seasonal_period=1, fig_kwargs=global_plot_settings, session_id=42)
exp.plot_model()

# Includes ACF, PACF & more ----
exp.plot_model(plot="diagnostics", fig_kwargs={"height": 900, "width": 1200})

,Description,Value
0,session_id,42
1,Target,Time Series
2,Original Data,"(1000, 1)"
3,Missing Values,False
4,Approach,Univariate
5,Exogenous Variables,Not Present
6,Transformed Train Target,"(999,)"
7,Transformed Test Target,"(1,)"
8,Transformed Train Exogenous,"(999, 0)"
9,Transformed Test Exogenous,"(1, 0)"


Intuition for identification:
* process memory: direct and indirect dependence information
* ACF for AR(p): strong until lag p, trailing off afterwards
* PACF only describes the direct relationship between an observation and its lag. This would suggest that there would be no correlation for lag values beyond k.
* analytical derivation is based on the Laurent series expansion and the backshift operator $B$


## MA processes

An autoregressive model expresses the forecast variable as a linear combination of past realizations of itself and the same idea can be applied to past forecast errors: a moving average process of order $q$ is defined by the relationship:
    
\begin{equation}
X_t = \epsilon_t + \sum_{i = 1}^q \theta_i \epsilon_{t-i}
\end{equation}

where $\epsilon_t$ is a white noise series.
  
Let's repeat the identification exercise from before:

\begin{equation}
    X_t = \epsilon_t + 0.9  \epsilon_{t-1}
\end{equation}


In [6]:
ar1 = np.array([1])
ma1 = np.array([1, -0.9])
MA_object1 = ArmaProcess(ar1, ma1)
simulated_data_1 = pd.Series(MA_object1.generate_sample(nsample=1000))

# #### Using Manual Approach ----
# plt.plot(simulated_data_1)
# plt.show()

# plot_acf(simulated_data_1, lags = 10)
# plt.show()

# plot_pacf(simulated_data_1, lags = 10)
# plt.show()

################################
#### Using PyCaret Approach ####
################################
exp = TSForecastingExperiment()
exp.setup(data = simulated_data_1, seasonal_period=1, fig_kwargs=global_plot_settings, session_id=42)
exp.plot_model()

# Includes ACF, PACF & more ----
exp.plot_model(plot="diagnostics", fig_kwargs={"height": 900, "width": 1200})

,Description,Value
0,session_id,42
1,Target,Time Series
2,Original Data,"(1000, 1)"
3,Missing Values,False
4,Approach,Univariate
5,Exogenous Variables,Not Present
6,Transformed Train Target,"(999,)"
7,Transformed Test Target,"(1,)"
8,Transformed Train Exogenous,"(999, 0)"
9,Transformed Test Exogenous,"(1, 0)"


Intuition for identification:
* MA model corrects future forecasts based on errors made on recent forecasts.
* ACF for MA(k) series: to show a strong correlation with recent values up to the lag of k, then a sharp decline 
* PACF: strong relationship to the lag k and a trailing off of correlation from the lag onwards.

## ARMA

We have an autoregressive component and a moving average one, so it is quite natural to combine those two types of dynamics into a single model: ARMA(p,q) series satisfies the relationship:

\begin{equation}
X_t = \sum_{i=1}^p \phi_i X_{t-i} + \sum_{j=1}^q \theta_j \epsilon_{t-j} + \epsilon_t
\end{equation}

where our predictors on the right hand side include both lagged values of the series and lagged errors, $p$ is the order the autoregressive part and $q$ is the order of the moving average component.

Some observations:
1. if an ARMA(p,q) model is stationary (see [this one](https://www.kaggle.com/konradb/ts-0-the-basics#Stationarity) for a refresher), it can be represented as an infinite AR series

\begin{equation}    
    X_t = \sum_{u=1}^\infty \pi_u X_{t-u} + \epsilon_t
\end{equation}

2. this allows for estimation via maximum likelihood and a simple recursive forecast:

\begin{equation}
    \hat{X}_{T+1} = \sum_{u=1}^\infty \hat{\pi}_u X_{T+1 - u}
\end{equation}

In practice: Kalman filter.

3. Stationarity of the ARMA(p,q) process is established by analyzing the characteristic polynomial. There is quite a substantial body of theory behind this idea, but we are focusing on practical applications here - so a crash-course argument would go like this:

    * given ARMA(p,q) series defined by the equation above, we can write its characteristic polynomial 
    
    \begin{equation}
    P(z) = 1 - \phi_1 z - \ldots - \phi_p z^p
    \end{equation}
    
    * if we look for solutions in complex (as opposed to real numbers) domain, the equation P(z) = 0 has $p$ solutions $z_1, \ldots, z_p$ (corollary from the Fundamental Theorem of Algebra: https://en.wikipedia.org/wiki/Fundamental_theorem_of_algebra)
       
    * if $|z_i| >1$ for all $i$ then the underlying model is stationary
    * tests like Augmented Dickey-Fuller check for this behavior
    

In [7]:

# Image(filename="../input/muh-images/unit_roots.png", width= 600, height=200)
 

## Forecasting with ARMA

Let's put our newly introduced framework to the test and actually predict something. We look we will have a look at the quarterly change in the aggregate savings level in the United States - the data is sourced from FRED: https://fred.stlouisfed.org/series/A191RP1Q027SBEA.

In [8]:
xdat = pd.read_csv("https://raw.githubusercontent.com/pycaret/public_examples/main/data/tsdata-1/savings_change.csv")

xdat.columns = ['date', 'value']
xdat['date'] = pd.to_datetime(xdat['date'])

We start with a quick EDA: first, a seasonal decomposition

In [9]:
# #### Using Manual Approach ----
# xdat.set_index('date').plot()

# decomposition = sm.tsa.seasonal_decompose(xdat["value"],period =12, model = 'additive') 
# figure = decomposition.plot()
# plt.show()

################################
#### Using PyCaret Approach ####
################################
exp = TSForecastingExperiment()
exp.setup(data = xdat, index="date", fh=12, fig_kwargs=global_plot_settings, session_id=42)

exp.plot_model()
exp.plot_model(plot="decomp")

# NOTE: Data is quarterly, hence a seasonality of 4 was tested.
# However, seasonality was not detected at this period, hence
# no seasonality is use for modeling and analysis. 

,Description,Value
0,session_id,42
1,Target,value
2,Original Data,"(187, 1)"
3,Missing Values,False
4,Approach,Univariate
5,Exogenous Variables,Not Present
6,Transformed Train Target,"(175,)"
7,Transformed Test Target,"(12,)"
8,Transformed Train Exogenous,"(175, 0)"
9,Transformed Test Exogenous,"(12, 0)"


There is *a* seasonal component, but it is very small in magnitude; trend does not seem significant either - we can suspect the process is stationary and move to testing this formally (using our old acquaintance, the ADF test):

In [10]:
# #### Using Manual Approach ----

# result = adfuller(xdat['value'])
# print('ADF Statistic: %f' % result[0])
# print('p-value: %f' % result[1])
# print('Critical Values:')
# for key, value in result[4].items():
#     print('\t%s: %.3f' % (key, value))

################################
#### Using PyCaret Approach ####
################################

exp.check_stats(test="adf")

# NOTE: Data is stationary, hence our original assessment that there is no seasonality is correct.

,Test,Test Name,Data,Property,Setting,Value
0,Stationarity,ADF,Actual,Stationarity,{'alpha': 0.05},True
1,Stationarity,ADF,Actual,p-value,{'alpha': 0.05},0.0
2,Stationarity,ADF,Actual,Test Statistic,{'alpha': 0.05},-18.70576
3,Stationarity,ADF,Actual,Critical Value 1%,{'alpha': 0.05},-3.466005
4,Stationarity,ADF,Actual,Critical Value 5%,{'alpha': 0.05},-2.877208
5,Stationarity,ADF,Actual,Critical Value 10%,{'alpha': 0.05},-2.575122


No red flags stopping us at this stage (reminder: this means we don't see an overall linear trend during the sample period) - we can proceed to model the series as ARMA(p,q). What about the values of p and q?

In [11]:
# #### Using Manual Approach ----
# plot_acf(xdat['value'], lags = 25); print()
# plot_pacf(xdat['value'], lags = 25); print()

################################
#### Using PyCaret Approach ####
################################

# We can use the "diagnotics" plot as before or just plot ACF and PACF separatelty as shown below
exp.plot_model(plot="acf")
exp.plot_model(plot="pacf")

At this stage, we could make an educated guess at fitting an ARMA model - (1,0) seems like a reasonable first pass. We can do something better though and use the functionality provided by the `pmdarima` package. Its functionality modernizes the ARIMA functionality from `statsmodels`:
* it allows working with pipelines combining preprocessing and fitting
* enables cross-validation functionality akin to what we saw with`cross_validation` in [Prophet](https://www.kaggle.com/konradb/ts-1b-prophet#Performance-evaluation)
* automates a (somewhat tedious) task of establishing the right order for an ARIMA model

In [12]:
# #### Using Manual Approach ----
# # we split the data into training and validation, leaving out the last 3 years as holdout
# xtrain, xvalid = model_selection.train_test_split(xdat['value'], test_size = 12)

################################
#### Using PyCaret Approach ####
################################
# NOTE: This is not needed separately as this was already done during setup.
# Refer to the details printed after setup for details.

One of the useful features of `pmdarima` is a Fourier featurizer:
- it allows us to capture seasonality in the model without worrying about the seasonal order (so use a pure ARIMA with `seasonality = False`) 
- its achieved by creating a set of covariates based on Fourier decomposition
- we can include arbitrary length seasonal pattern
- the short-term dynamics are easily handled with a simple ARMA error

In [13]:
# #### Using Manual Approach ----
# # construct a pipeline, combining feature preprocessing and the model to fit
# pipe = pipeline.Pipeline([
#     ("fourier", ppc.FourierFeaturizer(m=4)),
#     ("arima", arima.AutoARIMA(stepwise=True, trace=1, error_action="ignore",
#                               seasonal=False,  # because we use Fourier
#                               suppress_warnings=True))
# ])

# pipe.fit(xtrain)

################################
#### Using PyCaret Approach ####
################################

# Just specify "auto_arima". PyCaret will take care of the rest.
pipe = exp.create_model("auto_arima")

# Alternately, you can specify the parameters manually if you wish, e.g. 
# pipe = exp.create_model("auto_arima", stepwise=True, seasonal=False)

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,2004Q3,15.0707,19.2496,0.9469,1.7109,1.3025,1.0270,-0.0027
1,2007Q3,17.1388,23.2210,0.9255,1.8379,1.4091,1.2040,-0.0962
2,2010Q3,13.7325,22.9590,1.0272,1.6599,1.0206,1.0877,-0.0038
Mean,NaT,15.3140,21.8099,0.9665,1.7362,1.2440,1.1062,-0.0342
SD,NaT,1.4012,1.8135,0.0438,0.0748,0.1639,0.0735,0.0438


In [14]:
# we can inspect the results
print(pipe.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  175
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -701.306
Date:                Sun, 06 Mar 2022   AIC                           1408.612
Time:                        00:21:46   BIC                           1418.107
Sample:                             0   HQIC                          1412.464
                                - 175                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      1.5364      1.055      1.456      0.145      -0.531       3.604
ar.L1         -0.3105      0.042     -7.310      0.000      -0.394      -0.227
sigma2       177.0423     10.564     16.759      0.0

Statsmodels provides us with a rich set of descriptive measures characterizing the fitted model:
* the top table summarizes sample size, likelihood and the information criteria - this is helpful if we want to select models on parsimony. For those unfamiliar with the topic, information criteria (AIC, BIC, HQIC ) quantify the basic idea of balancing model complexity (number of parameters) and likelihood, so that we end up selecting a model which has the minimal number of parameters necessary to capture the details of our DGP, but not more than that:https://en.wikipedia.org/wiki/Akaike_information_criterion. An information criterion measures how well a model fits the data while taking into account the overall complexity - model that fits the data very well while using large number of parameters will be assigned a larger AIC score than a model that uses fewer features to achieve the same goodness-of-fit. 

* the middle table gives the estimated coefficients of the model: $ar.L_{i}$ corresponds to the coefficient of the AR part at lag $i$, which $sigma2$ is the variance of the noise component. The $Z$ statistic and the endpoints of a confidence interval allow for a quick assessment of statistical significance

* finally, the box at the bottom summarizes some diagnostic test: Jarque-Bera is used for testing if the residuals of the model have Gaussian distribution, while Ljung-Box checks whether the results are serially independent. In our case the former shows no problem, while L-B indicates there is some serial dependence in the residuals of our model, so there mighte be some component of the dynamics we are not capturing. 

In [15]:
# #### Using Manual Approach ----
# # we compute predictions from the pipeline object
# preds, conf_int = pipe.predict(n_periods= xvalid.shape[0], return_conf_int=True)
# print("\nForecasts:")
# print(preds)

# print("\nConfidence intervals:")
# print(conf_int)

################################
#### Using PyCaret Approach ####
################################
preds = exp.predict_model(pipe, return_pred_int=True)
preds

,Model,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,Auto ARIMA,4.3123,5.7126,1.221,1.4692,0.302,0.2511,0.0431


,y_pred,lower,upper
2013Q4,0.4515,-25.6272,26.5302
2014Q1,1.3962,-25.9108,28.7032
2014Q2,1.1029,-26.3197,28.5254
2014Q3,1.1940,-26.2397,28.6276
2014Q4,1.1657,-26.2690,28.6004
2015Q1,1.1745,-26.2604,28.6093
2015Q2,1.1717,-26.2631,28.6066
2015Q3,1.1726,-26.2623,28.6074
2015Q4,1.1723,-26.2625,28.6072
2016Q1,1.1724,-26.2624,28.6072


Unlike the uncertainty bands for Prophet (which are calculated non-parametrically by resampling the data with MCMC), ARMA confidence intervals are based on a parametric approach:
* we assume the residual terms $\epsilon_t$ have Gaussian distribution
* Gaussian is closed under addition $\implies$ $X_t$ is Gaussian
* Hilbert projection theorem $\implies$ expected value is our best forecast wrt MSE
* confidence interval for the mean: $\mu_t \pm 1.96 \; \sigma_t$ with
\begin{equation}
\mu_t = E\left[X_t | \sigma(X_t) \right]
\end{equation}
and
\begin{equation}
\sigma^2_t = Var\left[X_t | \sigma(X_t) \right]
\end{equation}

In [16]:
#### Using Manual Approach ----
# # let's quickly visualize the forecast
# xvalid = pd.DataFrame(xvalid.values, columns = ['actual'])
# xvalid['predicted'] = preds
# xvalid.plot()

################################
#### Using PyCaret Approach ####
################################
exp.plot_model(pipe)

In terms of forecast quality this model can be useful as a baseline, but overall it is clearly nothing to write home about:
1. there is a predictable lag: one period after an increase in the original series the forecast goes up as well; this is due to autoregressive nature of the model
2. the model captures the general dynamics of the model, but struggles with the range of values - this is a consequence of the constant variance assumption

<a id="section-two"></a>
# Beyond ARMA

As we can see from the examples above, there are issues when applying the baseline $ARMA(p,q)$ model to real life time series:
* the models are only efficient if the series is stationary - peformance deteriorates rapidly otherwise 
* does not allow for trend
* does not allow for seasonality

The last two issues can be handled by differencing the series at appropriate lags, and it is this idea that allows us to extend ARMA - first, we move towards ARIMA. 

### ARIMA

A process $X_t$ is ARIMA(p,d,q) $\iff$ $\nabla^d X_t$ is a stationary ARMA(p,q)

Unpacking the (slightly cryptic :-) formulation: 
* a series becomes a stationary ARMA(p,q) after differencing it $d$ times

* differencing is a discrete version of differentiation: polynomial of order $d$ becomes a constant after taking $d$th derivative $\implies$ a series stationary around a linear trend becomes stationary in first differences (quadratic trend - second differences etc)



In [17]:
# quick demo: Tesla daily closing price on NYSE 
xdat = pd.read_csv("https://raw.githubusercontent.com/pycaret/public_examples/main/data/tsdata-1/tesla_prices_5y.csv")
xdat = xdat[['Date', 'Close']]
xdat.columns = ['date', 'value']
xdat['date'] = pd.to_datetime(xdat['date'])
xdat.set_index('date', inplace=True)

# PyCaret currently does not work on Business Week Daily datasets.
# Hence resampling to Weekly Data.
# Follow for updates: https://github.com/pycaret/pycaret/issues/1854 
xdat = xdat.resample("W").mean()

Applying Dickey-Fuller test confirms that the series certainly is not stationary:

In [18]:
# #### Using Manual Approach ----
# # xdat['value'] = np.log1p(xdat['value'])
# xdat['date'] = pd.to_datetime(xdat['date'])
# xdat.set_index('date').plot()

# result = adfuller(xdat['value'])
# print('ADF Statistic: %f' % result[0])
# print('p-value: %f' % result[1])

# # With apologies for the pun, applying `.diff()` operator makes all the difference:

# ydat = xdat['value'].diff()
# ydat.plot()
# print()

# result = adfuller(ydat.dropna())
# print('ADF Statistic: %f' % result[0])
# print('p-value: %f' % result[1])

################################
#### Using PyCaret Approach ####
################################
exp = TSForecastingExperiment()
exp.setup(data = xdat, fh=4, fig_kwargs=global_plot_settings, session_id=42)

# Plot first difference (default)
exp.plot_model(plot="diff")

# Check ADF for original and first difference
exp.check_stats(test="adf", data_kwargs={"order_list":[1]})

,Description,Value
0,session_id,42
1,Target,value
2,Original Data,"(251, 1)"
3,Missing Values,False
4,Approach,Univariate
5,Exogenous Variables,Not Present
6,Transformed Train Target,"(247,)"
7,Transformed Test Target,"(4,)"
8,Transformed Train Exogenous,"(247, 0)"
9,Transformed Test Exogenous,"(4, 0)"


,Test,Test Name,Data,Property,Setting,Value
0,Stationarity,ADF,Actual,Stationarity,{'alpha': 0.05},False
1,Stationarity,ADF,Actual,p-value,{'alpha': 0.05},0.973711
2,Stationarity,ADF,Actual,Test Statistic,{'alpha': 0.05},0.226525
3,Stationarity,ADF,Actual,Critical Value 1%,{'alpha': 0.05},-3.458011
4,Stationarity,ADF,Actual,Critical Value 5%,{'alpha': 0.05},-2.87371
5,Stationarity,ADF,Actual,Critical Value 10%,{'alpha': 0.05},-2.573256
6,Stationarity,ADF,Order=1,Stationarity,{'alpha': 0.05},True
7,Stationarity,ADF,Order=1,p-value,{'alpha': 0.05},0.000094
8,Stationarity,ADF,Order=1,Test Statistic,{'alpha': 0.05},-4.673301
9,Stationarity,ADF,Order=1,Critical Value 1%,{'alpha': 0.05},-3.458011


As you can see, differencing the time series once removes a linear trend and produces a series for which the null is not rejected in Dickey-Fuller test - this is despite the fact the volatility clearly goes up in the latter part of the sample!

The basic idea is that if we are dealing with a series that can be represented as a stationary process with polynomial trend, we can model it jointly (instead of manually differencing $d$ times and then flipping back). 


Sideline point worth mentioning is that a non-seasonal ARIMA encapsulates other models as special cases (to quote a line from several textbooks I used in university: verifying those assertions is left as an exercise to the reader ;-)

- ARIMA(0,0,0) is white noise
- ARIMA(0,1,0) is random walk
- ARIMA(p,0,0) is AR(p)
- ARIMA(0,0,q) is MA(q)
- ARIMA(0, 1, 1) is equivalent to single (basic) exponential smoothing model
- ARIMA(0, 2, 2) is equivalent to Holt's linear method with additive errors, i.e. [double exponential smoothing](https://www.kaggle.com/konradb/ts-1a-smoothing-methods#Popular-methods)


### SARIMA

The "I" in ARIMA corresponds to the "integrated" component, which is a formal way of saying we are incorporating the trend into our setup starting with a stationary $ARMA(p,q)$ process. Following a similar logic, we can incorporate the seasonal component and allow it to follow the same type of dynamic: a seasonal ARIMA (SARIMA) model can be denoted as ARIMA(p,d,q)(P,D,Q)m, where:
- m refers to the number of periods in each season
- (lowercase) p,d,q refer to the definition of the ARIMA part
- (uppercase) P,D,Q refer to the autoregressive, differencing, and moving average terms for the seasonal part of the ARIMA model.



The last part is not entirely obvious at an intuitive model (at least it wasn't to me when I learned it). A good intuition to think about is that we want to identify components of the seasonal part the way we would for the series itself:
* we isolate the seasonal component, e.g. through seasonal decomposition
* to establish the value of $D$ we test the seasonal component - is it stationary? is the differenced version stationary? 
* the seasonal autoregressive value $P$ can be established by inspecting the PACF of the seasonal component
* the seasonal moving average value $Q$ can be established by inspect the ACF of the seasonal component

Below, we demonstrate briefly how this analysis can be conducted in practice:

In [19]:
#### Using Manual Approach ----
# series = pd.read_csv(CFG.data_folder + 'passengers.csv')
# series['date'] = pd.to_datetime(series['date'])

# decomposition = seasonal_decompose(series['passengers'], period = 12, model = 'multiplicative')
# figure = decomposition.plot()
# plt.show()

################################
#### Using PyCaret Approach ####
################################
from pycaret.datasets import get_data
series = get_data("airline")

exp = TSForecastingExperiment()
exp.setup(data = series, fh=12, fig_kwargs=global_plot_settings, session_id=42)
decompositon = exp.plot_model(plot="decomp", data_kwargs={"type": "multiplicative"}, return_data=True)

,Description,Value
0,session_id,42
1,Target,Number of airline passengers
2,Original Data,"(144, 1)"
3,Missing Values,False
4,Approach,Univariate
5,Exogenous Variables,Not Present
6,Transformed Train Target,"(132,)"
7,Transformed Test Target,"(12,)"
8,Transformed Train Exogenous,"(132, 0)"
9,Transformed Test Exogenous,"(12, 0)"


In [20]:
# #### Using Manual Approach ----
# # check stationarity
# result = adfuller(decomposition.seasonal)
# print('ADF Statistic: %f' % result[0])
# print('p-value: %f' % result[1])


################################
#### Using PyCaret Approach ####
################################
eda = TSForecastingExperiment()
eda.setup(data = decompositon.get("seasonal"), fh=12, fig_kwargs=global_plot_settings, session_id=42)

eda.check_stats(test="adf")

,Description,Value
0,session_id,42
1,Target,seasonal
2,Original Data,"(144, 1)"
3,Missing Values,False
4,Approach,Univariate
5,Exogenous Variables,Not Present
6,Transformed Train Target,"(132,)"
7,Transformed Test Target,"(12,)"
8,Transformed Train Exogenous,"(132, 0)"
9,Transformed Test Exogenous,"(12, 0)"


,Test,Test Name,Data,Property,Setting,Value
0,Stationarity,ADF,Actual,Stationarity,{'alpha': 0.05},True
1,Stationarity,ADF,Actual,p-value,{'alpha': 0.05},0.0
2,Stationarity,ADF,Actual,Test Statistic,{'alpha': 0.05},-2733833359999868.0
3,Stationarity,ADF,Actual,Critical Value 1%,{'alpha': 0.05},-3.482088
4,Stationarity,ADF,Actual,Critical Value 5%,{'alpha': 0.05},-2.884219
5,Stationarity,ADF,Actual,Critical Value 10%,{'alpha': 0.05},-2.578864


Non-stationarity is rejected extremely strongly, do $D$ = 0 in our example. What about the other components?

In [21]:
# #### Using Manual Approach ----
# plot_pacf(decomposition.seasonal, lags = 12); print()
# plot_acf(decomposition.seasonal, lags = 12); print()

################################
#### Using PyCaret Approach ####
################################
eda.plot_model(plot="pacf", data_kwargs={"nlags": 12})
eda.plot_model(plot="acf", data_kwargs={"nlags": 12})

Relevant PACF values up until lag 12 at least indicate $P$ = 1 is the right choice.

Based on the ACF graph above $Q=1$ seems like a reasonable choice.


For the more mathematically members of the audience, the SARIMA model can be formulated quite compactly if we make use of the backshift (lag) operator (https://en.wikipedia.org/wiki/Lag_operator).  The seasonal part of the model consists of terms that are similar to the non-seasonal components of the model, but involve backshifts of the seasonal period. For example, an $ARIMA(1,1,1)(1,1,1)_4$ model for quarterly data (m = 4) series $y_t$ can be specified as 

\begin{equation}       
    (1 - \phi_1 B)(1- \Phi_1 B^4)(1- B)(1-B^4)y_t = (1 + \theta_1 B)(1 + \Theta_1 B)\epsilon_t
\end{equation}

### SARIMAX

Sometimes we do have data available for the forecast horizon (e.g. long term economic forecasts available in advance, or the lagged values of the series itself). This means we can use **exogenous** variables to improve the quality of the forecast - hence SARIMA**X**.

<a id="section-three"></a>
# Full pipeline

We are in a position to combine all the building blocks and solve a real-life problem in full generality. We will use the data from the Demand Forecasting competition: 
https://www.kaggle.com/c/demand-forecasting-kernels-only

In [22]:
# load the data
train = pd.read_csv("https://raw.githubusercontent.com/pycaret/public_examples/main/data/demand-forecasting-kernels-only/train.csv", parse_dates=['date'], index_col='date')
test = pd.read_csv("https://raw.githubusercontent.com/pycaret/public_examples/main/data/demand-forecasting-kernels-only/test.csv", parse_dates=['date'],index_col='date')
df = pd.concat([train,test],sort=True)

In [23]:
# we subset to one item x store combination
buf = df[(df.item==1)&(df.store==1)].copy()
buf.head(10)

,id,item,sales,store
date,,,,
2013-01-01,NaN,1,13.0,1
2013-01-02,NaN,1,11.0,1
2013-01-03,NaN,1,14.0,1
2013-01-04,NaN,1,13.0,1
2013-01-05,NaN,1,10.0,1
2013-01-06,NaN,1,12.0,1
2013-01-07,NaN,1,10.0,1
2013-01-08,NaN,1,9.0,1
2013-01-09,NaN,1,12.0,1


In [24]:

# #### Using Manual Approach ----
# # what do the components look like? 
# decomposition = seasonal_decompose(buf.sales.dropna(),freq=365)
# figure = decomposition.plot()
# plt.show()

################################
#### Using PyCaret Approach ####
################################
exp = TSForecastingExperiment()
exp.setup(data = buf["sales"].dropna(), fh=90, fig_kwargs=global_plot_settings, session_id=42)
exp.plot_model(plot="decomp")

# NOTE: Konrad's original notebook uses a frequency of 365 in the decomposition
# PyCaret will use the detected frequency by default (i.e. 7 in this case).

,Description,Value
0,session_id,42
1,Target,sales
2,Original Data,"(1826, 1)"
3,Missing Values,False
4,Approach,Univariate
5,Exogenous Variables,Not Present
6,Transformed Train Target,"(1736,)"
7,Transformed Test Target,"(90,)"
8,Transformed Train Exogenous,"(1736, 0)"
9,Transformed Test Exogenous,"(90, 0)"


With four years worth of data, we can think about capturing seasonal patterns (always worth remembering: modeling seasonal effects makes sense if you have at least two complete data cycles in your dataset). We begin by splitting into training and test:

In [25]:
#### Using Manual Approach ----
# tr_start,tr_end = '2014-01-01','2017-09-30'
# te_start,te_end = '2017-10-01','2017-12-31'
# x0 = buf['sales'][tr_start:tr_end].dropna()
# x1 = buf['sales'][te_start:te_end].dropna()

# # examine autocorrelation
# plot_acf(x0, lags = 12); print()
# plot_pacf(x0, lags = 12); print()

################################
#### Using PyCaret Approach ####
################################

exp.plot_model(plot="acf")
exp.plot_model(plot="pacf")

Indeed it does. We will conduct a quick grid search across possible values for the model order coefficients. 

In [26]:
# #### Using Manual Approach ----
# model_autoARIMA = auto_arima(x0, start_p=7, start_q=7 ,
#                       test='adf',       
#                       max_p= 7, max_q=7, 
#                       m= 7,              
#                       d= 1,
#                       seasonal=True,   
#                       start_P=1, 
#                       D=1, 
#                       trace=True,
#                       error_action='ignore',  
#                       suppress_warnings=True, 
#                       stepwise=True)

################################
#### Using PyCaret Approach ####
################################
model_autoARIMA = exp.create_model("auto_arima")

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,2017-01-05,3.8336,4.8102,0.2493,0.2274,0.7978,0.7837,0.0524
1,2017-04-05,6.9709,8.7984,0.2446,0.2857,1.4511,1.4347,-0.8060
2,2017-07-04,8.3790,9.8364,0.3949,0.3003,1.7251,1.5859,-1.4517
Mean,NaT,6.3945,7.8150,0.2963,0.2711,1.3247,1.2681,-0.7351
SD,NaT,1.8999,2.1665,0.0698,0.0315,0.3890,0.3480,0.6161


In [27]:
print(model_autoARIMA.summary())

                                     SARIMAX Results                                     
Dep. Variable:                                 y   No. Observations:                 1736
Model:             SARIMAX(3, 1, 1)x(1, 0, 1, 7)   Log Likelihood               -5156.252
Date:                           Sun, 06 Mar 2022   AIC                          10328.505
Time:                                   00:32:43   BIC                          10372.175
Sample:                                        0   HQIC                         10344.655
                                          - 1736                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0003      0.001      0.331      0.741      -0.001       0.002
ar.L1          0.0371      0.028      1.342

In [28]:
# #### Using Manual Approach ----
# model_autoARIMA.plot_diagnostics()
# plt.show()

################################
#### Using PyCaret Approach ####
################################
exp.plot_model(model_autoARIMA, plot="diagnostics")

In [29]:
# #### Using Manual Approach ----
# pred = model_autoARIMA.predict(x1.shape[0])

# pd.DataFrame({'test':x1,'pred':pred}).plot();plt.show()

################################
#### Using PyCaret Approach ####
################################
exp.plot_model(model_autoARIMA)

As you can see from the graph above (and the diagnostics summary that preceded it), automatically fitted SARIMA does a decent job at capturing the dynamics of the underlying process - but there are still issues with capturing the range of values, with the most likely reasons being time-varying variance or distribution asymmetry. 


Let's compare its performance against [Prophet](https://www.kaggle.com/konradb/ts-1b-prophet).

<a id="section-four"></a>
# A day at the races

We will be using data from the Nifty-50 index, which is an aggregate for the National Stock Exchange in India (an updated dataset is maintained courtesy of [Rohan](https://www.kaggle.com/rohanrao)): https://www.kaggle.com/rohanrao/nifty50-stock-market-data. Both models will be fitted with default settings, without incorporating special days and the like.

In [30]:
# load the data
df = pd.read_csv("https://raw.githubusercontent.com/pycaret/public_examples/main/data/nifty50-stock-market-data/TATASTEEL.csv")
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)

# PyCaret currently does not work on Business Week Daily datasets.
# Hence resampling to Weekly Data.
# Follow for updates: https://github.com/pycaret/pycaret/issues/1854 
df = df.resample("W").mean()

df.head(5)

,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,
2000-01-09,154.080,155.010,163.44,151.8000,160.40,160.00,160.060,2720149.0,4.403860e+13,NaN,NaN,NaN
2000-01-16,169.280,169.370,173.76,162.5700,166.23,166.63,167.714,2362458.8,4.037360e+13,NaN,NaN,NaN
2000-01-23,166.850,168.510,173.83,163.4300,166.95,167.59,169.074,4102550.6,6.957630e+13,NaN,NaN,NaN
2000-01-30,152.575,153.325,155.60,146.9375,148.15,148.10,151.120,2234935.0,3.383948e+13,NaN,NaN,NaN
2000-02-06,141.080,141.940,144.03,139.1500,140.36,140.20,141.668,1864691.4,2.643813e+13,NaN,NaN,NaN


In [31]:
# #### Using Manual Approach ----
# # plot the target 
# df["VWAP"].plot()

################################
#### Using PyCaret Approach ####
################################
# We will plot this once we setup the experiment

In [32]:
# we can construct some simple features to be used as explanatory variables: 
# rolling statistics over the last 3 days for the price and volume characteristics

lag_features = ["High", "Low", "Volume"]
window_size = 3

df_rolled = df[lag_features].rolling(window = window_size, min_periods=0)
df_mean = df_rolled.mean().shift(1).reset_index()
df_std = df_rolled.std().shift(1).reset_index()

for feature in lag_features:
    df[feature + '_mean_lag' + str(window_size)] = df_mean[feature].values
    df[feature + '_std_lag' + str(window_size)] = df_std[feature].values

# Prophet can handle missing values, but ARIMA cannot
df.fillna(df.mean(), inplace=True)

In [33]:
# #### Using Manual Approach ----
# # prepare the training / validation split
# df_train = df[df.Date < "2019"]
# df_valid = df[df.Date >= "2019"]

################################
#### Using PyCaret Approach ####
################################
# We will pldo the split when we setup the experiment

exogenous_features = [
  'High_mean_lag3', 'High_std_lag3', 'Low_mean_lag3',
  'Low_std_lag3', 'Volume_mean_lag3', 'Volume_std_lag3'
]
target = "VWAP"   
selected_cols = [target] + exogenous_features

In [34]:
################################
#### Using PyCaret Approach ####
################################
exp = TSForecastingExperiment()
exp.setup(data = df[selected_cols], target=target, fh=121, fig_kwargs=global_plot_settings, session_id=42)
exp.plot_model(plot="train_test_split")

,Description,Value
0,session_id,42
1,Target,VWAP
2,Original Data,"(1113, 7)"
3,Missing Values,False
4,Approach,Univariate
5,Exogenous Variables,Present
6,Transformed Train Target,"(992,)"
7,Transformed Test Target,"(121,)"
8,Transformed Train Exogenous,"(992, 6)"
9,Transformed Test Exogenous,"(121, 6)"


In [35]:
# #### Using Manual Approach ----
# # fit ARIMA in the same manner as before 
# model_arima = auto_arima(df_train.VWAP, exogenous=df_train[exogenous_features], 
#                    m = 7,  
#                    # max ranges for the p,q,P,Q parameters - can be extended
#                    max_p= 2, max_q=2,    
#                    max_P = 1, max_Q = 1,
#                    trace=True, error_action="ignore", suppress_warnings=True)
# model_arima.fit(df_train.VWAP, exogenous=df_train[exogenous_features])

################################
#### Using PyCaret Approach ####
################################

# Disable cross_validation to match Konrad's flow.

# NOTE: This can take some time (~ 20 mins) since we are fitting multiple models
# with a seasonality of 52. More advanced users can try to build the model by hand
# by performing more EDA and specifying exact parameters to use. e.g. 
# exp.create_model("arima", order=(1,0,2))

# max ranges for the p,q,P,Q parameters - can be extended
model_arima = exp.create_model(
    "auto_arima",
    max_p= 2, max_q=2,
    max_P = 1, max_Q = 1,
    cross_validation = False
)

,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,26.0498,34.928,0.0578,0.0563,0.1778,0.1745,0.9436


In [36]:
#### Using Manual Approach ----
# forecast = model_arima.predict(n_periods=len(df_valid), exogenous=df_valid[exogenous_features])
# df_valid["Forecast_ARIMAX"] = forecast

################################
#### Using PyCaret Approach ####
################################
# Exogenous variables for Test are automatcally taken by PyCaret
forecast = exp.predict_model(model_arima)

,Model,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,Auto ARIMA,26.0498,34.928,0.0578,0.0563,0.1778,0.1745,0.9436


In [37]:
# #### Using Manual Approach ----
# # fit a Prophet model with default parameters
# df = df_train[['Date', 'VWAP']].rename(columns={"Date": "ds", "VWAP": "y"})

# model_prophet = Prophet()

# # add the regressors to the dataframe holding the data
# for f in exogenous_features:    
#     df[f] = df_train[f]
#     model_prophet.add_regressor(f)

# # the rest proceeds as before. 
# model_prophet.fit(df)

################################
#### Using PyCaret Approach ####
################################
model_prophet = exp.create_model("prophet", cross_validation=False)

,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,25.2064,33.6368,0.0572,0.0556,0.172,0.168,0.9477


In [38]:
# #### Using Manual Approach ----
# forecast = model_prophet.predict(df_valid[["Date", "VWAP"] + exogenous_features].rename(columns={"Date": "ds"}))
# df_valid["Forecast_Prophet"] = forecast.yhat.values

################################
#### Using PyCaret Approach ####
################################
# Exogenous variables for Test are automatcally taken by PyCaret
forecast = exp.predict_model(model_prophet)

,Model,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,Prophet,25.2064,33.6368,0.0572,0.0556,0.172,0.168,0.9477


In [39]:
print(model_arima.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  992
Model:               SARIMAX(1, 0, 2)   Log Likelihood               -4486.475
Date:                Sun, 06 Mar 2022   AIC                           8992.950
Time:                        00:51:43   BIC                           9041.948
Sample:                    01-09-2000   HQIC                          9011.580
                         - 01-06-2019                                         
Covariance Type:                  opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
High_mean_lag3       0.1843      0.131      1.403      0.161      -0.073       0.442
High_std_lag3        1.2915      0.120     10.764      0.000       1.056       1.527
Low_mean_lag3        0.8161 

In [40]:
# #### Using Manual Approach ----
# df_valid[["VWAP", "Forecast_ARIMAX", "Forecast_Prophet"]].plot()

################################
#### Using PyCaret Approach ####
################################
# In a future release, we will allow the ability to overlay model comparison 
exp.plot_model(model_arima)
exp.plot_model(model_prophet)


In [41]:
# #### Using Manual Approach ----
# def friendly_rmse(x,y):
#     return np.round(np.sqrt(mean_squared_error(x, y))  ,4)

# print("RMSE of Auto ARIMAX:", friendly_rmse(df_valid.VWAP, df_valid.Forecast_ARIMAX))
# print("RMSE of Prophet:", friendly_rmse(df_valid.VWAP, df_valid.Forecast_Prophet))

################################
#### Using PyCaret Approach ####
################################
# This was already provided when the models were built. No need to recompute this manually.
# Also, note that the metrics were rounded to 4 decimal places by default, but this can 
# be canged in the create_model call using the `round` argument.

As we can see from this small comparison, a vintage method like SARIMAX achieves comparable performance with a modern solution like Prophet. While it takes a bit more effort to set up, a full probabilistic model allows for more detailed and informative inference.

